In [ ]:
# default_exp core

# fastproaudio core

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

#from fastai.vision.all import *
from fastai.data.all import *
from subprocess import Popen, PIPE
import pyzenodo3
#from zipfile import ZipFile

# some data urls
URLs.AUDIOMDPI = 'https://zenodo.org/record/3562442'
URLs.MARCO = URLs.AUDIOMDPI  # just a shorthand alias I'm more likely to remember
URLs.SIGNALTRAIN_LA2A_1_1 = 'https://zenodo.org/record/3824876'
URLs.SIGNALTRAIN_LA2A_REDUCED = 'http://hedges.belmont.edu/data/SignalTrain_LA2A_Reduced.tgz'

In [ ]:
#export
def zenodo_url_to_data_url(url):
    #%pip install pyzenodo3 -q
    zen = pyzenodo3.Zenodo()
    record = url.split('/')[-1]
    return zen.get_record(record).data['files'][0]['links']['self']  

In [ ]:
zenodo_url_to_data_url(URLs.MARCO)

'https://zenodo.org/api/files/d6589bb4-d6a6-4bc6-8e51-e6334fafbe3f/AudioMDPI.zip'

In [ ]:
zenodo_url_to_data_url(URLs.SIGNALTRAIN_LA2A_1_1)

'https://zenodo.org/api/files/df302f12-7355-452e-93d1-b0c9344608f7/SignalTrain_LA2A_Dataset_1.1.tgz'

In [ ]:
#export 

# extract_func no longer supported  in untar_data, so zipfile stuff is unused
# thanks KevinB for the zip_extract! https://forums.fast.ai/t/generalizing-untar-data-to-also-work-with-zips/53741/14?u=drscotthawley
#def zip_extract(fname, dest):
#    zipfile.ZipFile(fname, mode='r').extractall(dest)

def get_audio_data(url):
    if ('zenodo' in url.lower()):
        url = zenodo_url_to_data_url(url)
    if '.zip' in url[-5:]:
        return untar_data(url)#, extract_func=zip_extract) #
    else:
        return untar_data(url) 

In [ ]:
#slow
path = get_audio_data(URLs.SIGNALTRAIN_LA2A_REDUCED)
path

Path('/home/shawley/.fastai/data/SignalTrain_LA2A_Reduced')

In [ ]:
#slow
path = get_audio_data(URLs.MARCO)
path

Path('/home/shawley/.fastai/data/AudioMDPI')